In [1]:
import numpy as np
import pandas as pd
from py_vollib_vectorized import get_all_greeks
import plotly as py
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode
init_notebook_mode(connected = True)

In [19]:
# 牛市看涨
# call：平值多头 + 虚值空头

s = 1000
k1,k2 = 1000,1500
t = 0.5
sigma = 0.3

# pnl
pnl = []
for i in range(0,2000,100):
    pnl1 = (i - k1 ) if i>k1 else 0
    pnl2 = 0 if i<k2 else (k2-i)
    pl = pnl1 + pnl2
    pnl.append(pl)

# delta
delta=[]
for i in range(0,2000,100):
    delta.append(get_all_greeks('c',i,k1,t,0,sigma)['delta'][0]-get_all_greeks('c',i,k2 ,t,0,sigma)['delta'][0])

# Gamma, Theta, Vega
gamma,theta,vega = [],[],[]
for i in range(0,2000,100):
    gamma.append(get_all_greeks('c',i,k1,t,0,sigma)['gamma'][0]-get_all_greeks('c',i,k2 ,t,0,sigma)['gamma'][0])
    theta.append(get_all_greeks('c',i,k1,t,0,sigma)['theta'][0]-get_all_greeks('c',i,k2 ,t,0,sigma)['theta'][0])
    vega.append(get_all_greeks('c',i,k1,t,0,sigma)['vega'][0]-get_all_greeks('c',i,k2 ,t,0,sigma)['vega'][0])

In [21]:
fig = make_subplots(3,1)
x = np.linspace(0,2000,100)

trace0 = go.Scatter(x = x,
                    y = pnl,
                    mode='lines',
                    name='pnl')
trace1 = go.Scatter(x = x,
                    y = delta,
                    mode='lines',
                    name='delta')
trace2 = go.Scatter(x = x,
                    y = gamma,
                    mode='lines',
                    name='gamma')
trace3 = go.Scatter(x = x,
                    y = theta,
                    mode='lines',
                    name='theta')
trace4 = go.Scatter(x = x,
                    y = gamma,
                    mode='lines',
                    name='vega')
fig.add_trace(trace0,1,1)
fig.add_trace(trace1,2,1)
fig.add_trace(trace2,3,1)
fig.add_trace(trace3,3,1)
fig.add_trace(trace4,3,1)

py.offline.iplot(fig)

In [18]:
get_all_greeks('c',i,k1,t,0,sigma)

,delta,gamma,theta,rho,vega
0,0.999131,0.000007,-0.003231,4.991252,0.040163
